In [1]:
import json
import time
import io
import pandas as pd


from kafka import KafkaProducer

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, window, col
from pyspark.sql import functions as F
import os

In [2]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

In [3]:
server = '35.197.53.150:9092'

In [4]:
producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

In [5]:
producer.bootstrap_connected()

True

In [ ]:
t0 = time.time()

In [ ]:
topic_name = 'test-topic'

In [ ]:
for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

In [ ]:
t1 = time.time()

In [ ]:
producer.flush()

In [ ]:
t2 = time.time()

In [ ]:
print(f'Sending took {(t1 - t0):.2f} seconds')

In [ ]:
print(f'Flush took {(t2 - t1):.2f} seconds')

In [ ]:
#on ssh run
!docker exec -it redpanda-1 rpk topic consume test-topic

In [ ]:
#Taxi Data

In [ ]:
#One time run

In [ ]:
url='https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz'

In [ ]:
!wget {url}

In [ ]:
!gunzip green_tripdata_2019-10.csv.gz

In [ ]:
#Create topic green-trips
!docker exec -it redpanda-1 rpk topic create green-trips

In [ ]:
#Rerun

In [6]:
filename='green_tripdata_2019-10.csv'

In [7]:
df_green = pd.read_csv(filename, usecols=['lpep_pickup_datetime','lpep_dropoff_datetime','PULocationID','DOLocationID','passenger_count','trip_distance','tip_amount'])

In [8]:
df_green.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26


In [ ]:
!docker exec -it redpanda-1 rpk topic list

In [ ]:
#Producer

In [ ]:
#t3 = time.time()

In [10]:
topic_name = 'green-trips'

In [11]:
t3 = time.time()
for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    #print(row_dict)
    #break
    #producer.send(topic_name, json.dumps(row_dict).encode('utf-8'))
    producer.send(topic_name, value=row_dict)
    #value=row_dict)
    #time.sleep(0.005)
    # TODO implement sending the data here
t4 = time.time()
print(f'Sending took {(t4 - t3):.2f} seconds')

Sending took 46.84 seconds


In [ ]:
#on ssh run
!docker exec -it redpanda-1 rpk topic consume green-trips

In [12]:
t5 = time.time()
producer.flush()
t6 = time.time()
print(f'Flushing took {(t6 - t5):.2f} seconds')

Flushing took 0.00 seconds


In [ ]:
#Consumer

In [13]:
#import pyspark
#from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate() 

:: loading settings :: url = jar:file:/home/Haitham.hamad/spark/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/Haitham.hamad/.ivy2/cache
The jars for the packages stored in: /home/Haitham.hamad/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-56db6a8a-1a57-4dd9-b051-fcebc70c1d5e;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in c

24/03/19 11:23:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [14]:
    #.option("startingOffsets", "earliest") \
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", server) \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [15]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/03/19 11:23:15 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-68139be4-0aad-4da4-9603-676c8dc5bb3b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/19 11:23:15 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:00:48", "lpep_dropoff_datetime": "2019-10-01 00:05:42", "PULocationID": 210, "DOLocationID": 108, "passenger_count": 2.0, "trip_distance": 1.03, "tip_amount": 2.19}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 19, 11, 21, 32, 136000), timestampType=0)


In [ ]:
spark.sparkContext.setLogLevel("DEBUG")

In [ ]:
spark.sparkContext.setLogLevel("OFF")

In [ ]:
query.stop()

In [16]:
parsed_df = green_stream \
    .selectExpr("CAST(value AS STRING)") \
    .select(F.from_json("value", "DOLocationID INT, PULocationID INT, lpep_pickup_datetime STRING, lpep_dropoff_datetime STRING, passenger_count INT, tip_amount DOUBLE, trip_distance DOUBLE") \
    .alias("data")) \
    .select("data.*")

In [ ]:
# Parse the stream data (assuming the data has appropriate columns)
#parsed_data = green_stream.selectExpr("CAST(value AS STRING) as raw_data")
parsed_df = green_stream.selectExpr("CAST(value AS STRING)") \
    .select(F.from_json("value", "DOLocationID INT") \
    .alias("data")) \
    .select("data.*")

In [17]:
# Add a timestamp column, group by timestamp and DOLocationID and order by count
result_df = parsed_df \
    .withColumn("timestamp", F.current_timestamp()) \
    .groupBy(F.window("timestamp", "5 minutes"), "DOLocationID") \
    .count() \
    .orderBy(F.desc("count"))

In [18]:
# Print the output to the console
query = result_df \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

24/03/19 11:23:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-dec1c93b-8062-4668-9c3a-0d5fbbe89996. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/19 11:23:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+------------+-----+
|              window|DOLocationID|count|
+--------------------+------------+-----+
|{2024-03-19 11:20...|          74|17733|
|{2024-03-19 11:20...|          42|15935|
|{2024-03-19 11:20...|          41|14058|
|{2024-03-19 11:20...|          75|12837|
|{2024-03-19 11:20...|         129|11922|
|{2024-03-19 11:20...|           7|11527|
|{2024-03-19 11:20...|         166|10843|
|{2024-03-19 11:20...|         236| 7913|
|{2024-03-19 11:20...|         223| 7540|
|{2024-03-19 11:20...|         238| 7318|
|{2024-03-19 11:20...|          82| 7285|
|{2024-03-19 11:20...|         181| 7279|
|{2024-03-19 11:20...|          95| 7244|
|{2024-03-19 11:20...|         244| 6731|
|{2024-03-19 11:20...|          61| 6605|
|{2024-03-19 11:20...|         116| 6337|
|{2024-03-19 11:20...|         138| 6144|
|{2024-03-19 11:20...|          97| 6049|
|{2024-03-19 11:20...

In [ ]:
# Wait for the query to terminate
query.awaitTermination()

In [ ]:
# Wait for the query to terminate or until the specified timeout
query.awaitTermination(timeout=60)  # Wait for 60 seconds

In [ ]:
# Stop the streaming query gracefully
query.stop()

In [ ]:
# Wait until the query is complete
while query.isActive:
    # Check if the query is still active
    if not query.isActive:
        break

# Stop the SparkSession
spark.stop()

In [19]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [20]:
parsed_green_stream = green_stream.selectExpr("CAST(value AS STRING)") \
    .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
    .select("data.*")

In [21]:
query = parsed_green_stream \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

24/03/19 11:25:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f5976c79-0048-498d-b35e-5811cec1d47c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/19 11:25:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+---------------------+------------+------------+---------------+-------------+----------+
|lpep_pickup_datetime|lpep_dropoff_datetime|PULocationID|DOLocationID|passenger_count|trip_distance|tip_amount|
+--------------------+---------------------+------------+------------+---------------+-------------+----------+
| 2019-10-01 00:00:48|  2019-10-01 00:05:42|         210|         108|            2.0|         1.03|      2.19|
| 2019-10-01 00:45:08|  2019-10-01 01:04:28|          83|          36|            2.0|         5.75|       0.0|
| 2019-10-01 00:32:44|  2019-10-01 00:46:53|          92|         260|            2.0|         6.01|      4.16|
| 2019-10-01 00:05:09|  2019-10-01 00:18:34|          75|         119|            1.0|         1.08|      2.36|
| 2019-10-01 00:18:16|  2019-10-01 00:53:16|          29|          92|            1.0|        21.39|       0.0|
| 2019-